In [1]:
import numpy as np

## Create synthetic data

In [2]:
nrows = 2**18
ncols = 60
grid = np.linspace(0,1.0, nrows)
features = np.empty((nrows,ncols))

for i in range(ncols):
    a = np.random.rand()
    b = np.random.rand()
    c = np.random.rand()
    feat = a*grid + b + np.random.normal(loc=0, scale=c, size=grid.size)
    features[:,i] = feat
    
response = np.empty(nrows)
a = np.random.rand()
b = np.random.rand()
c = np.random.rand()
response = a*grid + b + np.random.normal(loc=0, scale=c, size=grid.size)

## Split data and create C3 Datasets

In [3]:
from sklearn.model_selection import train_test_split

datasets = train_test_split(features, response, test_size=0.1, random_state=42)
X_train = c3.Dataset.fromPython(datasets[0])
X_test = c3.Dataset.fromPython(datasets[1])
y_train = c3.Dataset.fromPython(datasets[2])
y_test = c3.Dataset.fromPython(datasets[3])

## Build pipeline

In [4]:
linearRegression = c3.SklearnPipe(
                        name="linearRegression",
                        technique=c3.SklearnTechnique(
                            name="linear_model.LinearRegression",
                            # This tells ML pipeline to call "predict" method on sklearn.linear_model.LogisticRegression when we invoke the C3 action `process()` later.
                            processingFunctionName="predict"
                        )
                     )

In [5]:
lrPipeline = c3.MLSerialPipeline(
                name="lrPipeline",
                steps=[c3.MLStep(name="linearRegression",
                                 pipe=linearRegression)],
                scoringMetrics=c3.MLScoringMetric.toScoringMetricMap(scoringMetricList=[c3.MLRSquaredMetric()])
             )

## Train model

In [6]:
trainedLr = lrPipeline.train(input=X_train, targetOutput=y_train)

## Score model

In [7]:
scoreGr = trainedLr.score(input=X_test, targetOutput=y_test)
scoreGr

c3.Mapp<string, double>({'MLRSquaredMetric': 0.7350291972854365})

In [9]:
scoreGr['MLRSquaredMetric']

0.7350291972854365